In [1]:
import cv2
print(cv2.__version__)  # Should say 4.2.0
try:
    lsd = cv2.createLineSegmentDetector(0)
    print("LSD is available!")
except AttributeError:
    print("LSD not found in this OpenCV version.")

4.9.0
LSD is available!


In [2]:
import cv2
import numpy as np

# Read the image
img = cv2.imread("warped_image.png", cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError("Could not load warped_image.png")

# Threshold if not already binary (optional, skip if your IPM output is binary)
_, binary_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)

# Create LSD detector
lsd = cv2.createLineSegmentDetector(0)  # 0 = no refinement, tweak as needed

# Detect lines
lines = lsd.detect(binary_img)[0]  # Returns (lines, widths, precisions, nfa), we want lines
if lines is None:
    print("No lines detected.")
    lines = []


# Convert to cv::Vec4i-like format (x1, y1, x2, y2)
lines_vec4i = []
for line in lines:
    x1, y1, x2, y2 = map(int, line[0])  # line[0] is [x1, y1, x2, y2]
    lines_vec4i.append([x1, y1, x2, y2])

# Filter by length (e.g., min_length = 800 pixels)
min_length = 800
filtered_lines = []
for line in lines_vec4i:
    length = np.sqrt((line[2] - line[0])**2 + (line[3] - line[1])**2)
    if length >= min_length:
        filtered_lines.append(line)

# Limit to 2 longest lines (like your RANSAC)
filtered_lines.sort(key=lambda x: np.sqrt((x[2] - x[0])**2 + (x[3] - x[1])**2), reverse=True)
if len(filtered_lines) > 2:
    filtered_lines = filtered_lines[:2]

# Draw lines for visualization
output_img = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)
for line in filtered_lines:
    x1, y1, x2, y2 = line
    cv2.line(output_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    print("wow")

# Save and show result
cv2.imwrite("lsd_output.png", output_img)
cv2.imshow("LSD Lines", output_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(f"Detected {len(filtered_lines)} lines: {filtered_lines}")

Detected 0 lines: []


In [7]:
import cv2
import time
import numpy as np

# Load the image
img = cv2.imread("warped_image.png", cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError("Could not load warped_image.png")

# Create LSD detector
lsd = cv2.createLineSegmentDetector(0)

# Time the detection
start_time = time.time()
lines, widths, precisions, nfa = lsd.detect(img)  # Unpack all outputs
end_time = time.time()

# Handle case where no lines are detected
if lines is None:
    lines = []
    widths = []
    num_lines = 0
else:
    num_lines = len(lines)

# Print results with widths
print(f"Detected {num_lines} lines in {(end_time - start_time) * 1000:.2f} ms")
for i in range(min(5, num_lines)):
    x1, y1, x2, y2 = map(int, lines[i][0])
    width = widths[i][0] if widths is not None else 0.0
    print(f"Line {i}: ({x1}, {y1}) -> ({x2}, {y2}), Width: {width:.2f} pixels")

# Draw lines on a color image with unique colors
output_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
for i in range(num_lines):
    x1, y1, x2, y2 = map(int, lines[i][0])
    # Generate a color based on index (cycling through hues)
    hue = (i * 137 % 360)  # 137 is a prime number for varied hues
    color = tuple(int(c * 255) for c in cv2.cvtColor(
        np.array([[[hue, 1.0, 1.0]]], dtype=np.float32), cv2.COLOR_HSV2BGR)[0][0])
    cv2.line(output_img, (x1, y1), (x2, y2), color, 2)

# Write output image (overwrites existing file)
cv2.imwrite("lsd_output.png", output_img)
print("Wrote lsd_output.png")

Detected 20 lines in 36.54 ms
Line 0: (33, 921) -> (59, 1621), Width: 7.40 pixels
Line 1: (65, 1003) -> (58, 886), Width: 5.85 pixels
Line 2: (60, 1624) -> (79, 2173), Width: 7.50 pixels
Line 3: (78, 2174) -> (82, 2308), Width: 5.05 pixels
Line 4: (107, 2535) -> (103, 2404), Width: 5.53 pixels
Wrote lsd_output.png
